In [ ]:
from gurobipy import Model, GRB
import pandas as pd
from minutes_convertor import time_to_minutes

# Charger les données
chantiers_df = pd.read_excel('/Users/thomasdesforges/Desktop/CS/2A/ST7/Projet ST7/instance_WPY_realiste_jalon1.xlsx', sheet_name='Chantiers')
machines_df = pd.read_excel('/Users/thomasdesforges/Desktop/CS/2A/ST7/Projet ST7/instance_WPY_realiste_jalon1.xlsx', sheet_name='Machines')
sillons_arrivee_df = pd.read_excel('/Users/thomasdesforges/Desktop/CS/2A/ST7/Projet ST7/instance_WPY_realiste_jalon1.xlsx', sheet_name='Sillons arrivee')

# Définir un modèle Gurobi
model = Model("planification_taches_machines")

# Variables de décision : x_m,h,w = 1 si la machine m travaille sur le wagon w à l'heure h, 0 sinon
# Listes des machines, heures et wagons
machines = machines_df['Machine'].tolist()
wagons = sillons_arrivee_df['n°TRAIN'].tolist()
hours = range(24 * 60)  # De 0 à 1440 minutes (pour chaque minute de la journée)

# Créer des variables de décision pour chaque machine, heure et wagon
x = {}
for m in machines:
    for h in hours:
        for w in wagons:
            x[(m, h, w)] = model.addVar(vtype=GRB.BINARY, name=f"x_{m}_{h}_{w}")

# Affichage des variables créées (facultatif pour vérification)
print(f"Variables de décision créées : {len(x)}")

# Contrainte 1: Chaque tâche doit être attribuée à une machine et à un wagon à un créneau horaire spécifique
# Par exemple, la machine 'DEB' doit être affectée à un wagon à un moment donné
for _, row in machines_df.iterrows():
    task_type = row['Type de tache']
    unavailable_times = row['Indisponibilites']
    
    if unavailable_times != 0:
        periods = unavailable_times.strip('()').split(';')
        for period in periods:
            day_of_week, time_str = period.split(',')
            start_time = time_to_minutes(time_str.split('-')[0])
            end_time = time_to_minutes(time_str.split('-')[1])
            
            # Contrainte de disponibilité de la machine pendant les périodes d'indisponibilité
            for h in range(start_time, end_time):
                for w in wagons:
                    model.addConstr(sum(x[(m, h, w)] for m in machines if m == task_type) == 0)

# Contrainte 2: Chaque wagon doit avoir une machine affectée à un moment donné
for h in hours:
    for w in wagons:
        # La machine ne peut travailler que sur un wagon à un moment donné
        model.addConstr(sum(x[(m, h, w)] for m in machines) <= 1)

# Contrainte 3: Capacité des chantiers (respecter le nombre de voies disponibles)
# Modifier cette contrainte pour que le wagon soit attribué à un chantier spécifique
for _, row in chantiers_df.iterrows():
    chantier = row['Chantier']
    voies_disponibles = row['Nombre de voies']
    
    # Filtrer les wagons associés à ce chantier
    wagons_in_chantier = [w for w in wagons if w == chantier]  # Exemple de correspondance, à ajuster selon vos données

    # Calculer le nombre de wagons occupés sur chaque chantier à un moment donné
    for h in hours:
        model.addConstr(sum(x[(m, h, w)] for m in machines for w in wagons_in_chantier) <= voies_disponibles)

    
# Contrainte 4: Parallélisation des tâches humaines et machines (délai de 60 minutes entre arrivée et débranchement)
for _, row in sillons_arrivee_df.iterrows():
    train = row['n°TRAIN']
    model.addConstr(row['HDEP']-row['HARR']>=(60+15+15+150+15+20)) #diff_dep_arr>=(60+15+15+150+15+20))
    
    arrival_time = time_to_minutes(row['HARR'])
    
    # S'assurer que l'heure ne dépasse pas 1439 minutes
    if arrival_time + 60 <= 1439:  # Vérification que l'heure de débranchement reste dans la journée
        # Permettre à n'importe quelle machine de travailler sur le wagon, pas seulement "DEB"
        model.addConstr(sum(x[('DEB', arrival_time + 60, train)] for m in machines) == 1)
                  
# Contrainte 5: Parallélisation des tâches humaines et machines (délai de 150 minutes entre formation et dégarage)
for _, row in sillons_arrivee_df.iterrows():
    train = row['n°TRAIN']
    departure_time = time_to_minutes(row['HARR'])
    
    # S'assurer que l'heure ne dépasse pas 1439 minutes
    if arrival_time + 150 <= 1439:  # Vérification que l'heure de débranchement reste dans la journée
        # Permettre à n'importe quelle machine de travailler sur le wagon, pas seulement "DEB"
        model.addConstr(sum(x[('DEG', arrival_time + 60, train)] for m in machines) == 1)
          
# Contrainte 6: Parallélisation des tâches humaines et machines (délai de 20 minutes entre dégarage et départ)
for _, row in sillons_arrivee_df.iterrows():
    train = row['n°TRAIN']
    departure_time = time_to_minutes(row['HDEP'])
    
    # S'assurer que l'heure ne dépasse pas 1439 minutes
    if departure_time + 20 <= 1439:  # Vérification que l'heure de débranchement reste dans la journée
        # Permettre à n'importe quelle machine de travailler sur le wagon, pas seulement "DEB"
        model.addConstr(sum(x[('FOR', arrival_time + 60, train)] for m in machines) == 1)

# Optimisation
model.setObjective(0, GRB.MINIMIZE)  # Pas d'optimisation spécifique ici pour ce jalon

# Résolution du problème
model.optimize()

# Afficher les résultats
if model.status == GRB.OPTIMAL:
    for (m, h, w), var in x.items():
        if var.x > 0.5:  # Si la machine travaille sur le wagon à cette heure
            print(f"Machine {m} travaille sur le wagon {w} à la minute {h}")
else:
    print("Aucune solution optimale trouvée.")

Variables de décision créées : 164160
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.1.0 22A380)

CPU model: Intel(R) Core(TM) i5-7267U CPU @ 3.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 643784 rows, 479520 columns and 479832 nonzeros
Model fingerprint: 0xd075c598
Variable types: 0 continuous, 479520 integer (479520 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.21 seconds (0.03 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Machine DEB travaille sur le wagon 44228 à la minute 133
Machine DEB travaille sur le wagon 47240 à la minute 166